<a href="https://colab.research.google.com/github/ramizcihe/week4-cihe240058/blob/main/week4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
"""1. Dataset Preparation
Import the cyberbullying dataset (e.g., from Kaggle or other public sources).
Explore the dataset structure, label distribution, and check for class imbalance.
Clean the dataset by removing:
Missing or null values
Duplicate entries
Irrelevant content (e.g., ads, unrelated text)
"""

'1. Dataset Preparation\nImport the cyberbullying dataset (e.g., from Kaggle or other public sources).\nExplore the dataset structure, label distribution, and check for class imbalance.\nClean the dataset by removing:\nMissing or null values\nDuplicate entries\nIrrelevant content (e.g., ads, unrelated text)\n'

In [15]:
import pandas as pd

# Load dataset (replace with your dataset path or URL)
df = pd.read_csv("https://raw.githubusercontent.com/ramizcihe/week4-cihe240058/refs/heads/main/cyberbullying_tweets.csv")

# Show first 5 rows
df.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [16]:
print(df.columns)

Index(['tweet_text', 'cyberbullying_type'], dtype='object')


In [17]:
df['cyberbullying_type'].value_counts()

,count
cyberbullying_type,
religion,7998
age,7992
gender,7973
ethnicity,7961
not_cyberbullying,7945
other_cyberbullying,7823


In [18]:
X = df['tweet_text']                # features
y = df['cyberbullying_type']        # target

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [20]:
# Remove rows with null values
df = df.dropna()

print("After removing nulls:", df.shape)

After removing nulls: (47692, 2)


In [21]:
# Remove duplicate rows
df = df.drop_duplicates()

print("After removing duplicates:", df.shape)

After removing duplicates: (47656, 2)


In [22]:
df['tweet_text']

,tweet_text
0,"In other words #katandandre, your food was cra..."
1,Why is #aussietv so white? #MKR #theblock #ImA...
2,@XochitlSuckkks a classy whore? Or more red ve...
3,"@Jason_Gio meh. :P thanks for the heads up, b..."
4,@RudhoeEnglish This is an ISIS account pretend...
...,...
47687,"Black ppl aren't expected to do anything, depe..."
47688,Turner did not withhold his disappointment. Tu...
47689,I swear to God. This dumb nigger bitch. I have...
47690,Yea fuck you RT @therealexel: IF YOURE A NIGGE...


In [23]:
# Correct for your dataset
X = df['tweet_text']
y = df['cyberbullying_type']

In [24]:
"""
2. Text Preprocessing
Lowercase all text.
Remove stopwords, punctuation, numbers, and special characters.
Tokenize the text into words.
Apply stemming or lemmatization.
Optionally remove very rare or very frequent words to reduce noise
"""

'\n2. Text Preprocessing\nLowercase all text.\nRemove stopwords, punctuation, numbers, and special characters.\nTokenize the text into words.\nApply stemming or lemmatization.\nOptionally remove very rare or very frequent words to reduce noise\n'

In [26]:
['tweet_text', 'cyberbullying_type']

['tweet_text', 'cyberbullying_type']

In [29]:
df['tweet_text']          # ✅ correct features
df['cyberbullying_type']  # ✅ correct labels

,cyberbullying_type
0,not_cyberbullying
1,not_cyberbullying
2,not_cyberbullying
3,not_cyberbullying
4,not_cyberbullying
...,...
47687,ethnicity
47688,ethnicity
47689,ethnicity
47690,ethnicity


In [31]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')  # new requirement in latest NLTK
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [32]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter

# Download NLTK resources (only first time)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# 1. Lowercase
df['clean_text'] = df['tweet_text'].str.lower()

# 2. Remove punctuation, numbers, special characters
df['clean_text'] = df['clean_text'].apply(lambda x: re.sub(r'[^a-z\s]', '', x))

# 3. Tokenize
df['tokens'] = df['clean_text'].apply(word_tokenize)

# 4. Remove stopwords
stop_words = set(stopwords.words('english'))
df['tokens'] = df['tokens'].apply(lambda x: [word for word in x if word not in stop_words])

# 5. Lemmatization
lemmatizer = WordNetLemmatizer()
df['tokens'] = df['tokens'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

# 6. (Optional) Remove rare and frequent words
all_words = [word for tokens in df['tokens'] for word in tokens]
word_freq = Counter(all_words)

rare_thresh = 2      # words appearing less than 2 times
freq_thresh = 0.9    # words appearing in >90% of documents

rare_words = {w for w, c in word_freq.items() if c < rare_thresh}
total_docs = len(df)
freq_words = {w for w, c in word_freq.items() if c/total_docs > freq_thresh}

df['tokens'] = df['tokens'].apply(lambda x: [word for word in x if word not in rare_words and word not in freq_words])

# 7. (Optional) Reconstruct tokens back into cleaned sentence
df['final_text'] = df['tokens'].apply(lambda x: " ".join(x))

# Check sample
print(df[['tweet_text','final_text','cyberbullying_type']].head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


                                          tweet_text  \
0  In other words #katandandre, your food was cra...   
1  Why is #aussietv so white? #MKR #theblock #ImA...   
2  @XochitlSuckkks a classy whore? Or more red ve...   
3  @Jason_Gio meh. :P  thanks for the heads up, b...   
4  @RudhoeEnglish This is an ISIS account pretend...   

                                          final_text cyberbullying_type  
0                          word katandandre food mkr  not_cyberbullying  
1  white mkr theblock imacelebrityau today sunris...  not_cyberbullying  
2                    classy whore red velvet cupcake  not_cyberbullying  
3  jasongio meh p thanks head concerned another a...  not_cyberbullying  
4  rudhoeenglish isi account pretending kurdish a...  not_cyberbullying  


In [33]:
"""
3. Feature Extraction & Selection
Represent text as numerical features using:
TF-IDF vectors (word-level, n-grams, or character-level)
Word embeddings (Word2Vec, GloVe, FastText)
Transformer-based embeddings (BERT, DistilBERT)
Select relevant features (e.g., Chi-Square test, mutual information) to improve efficiency.
"""

'\n3. Feature Extraction & Selection\nRepresent text as numerical features using:\nTF-IDF vectors (word-level, n-grams, or character-level)\nWord embeddings (Word2Vec, GloVe, FastText)\nTransformer-based embeddings (BERT, DistilBERT)\nSelect relevant features (e.g., Chi-Square test, mutual information) to improve efficiency.\n'

In [36]:
# TF-IDF Vectorization
from sklearn.feature_extraction.text import TfidfVectorizer

# Word-level TF-IDF
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1,2))  # unigrams + bigrams
X_tfidf = tfidf.fit_transform(df['final_text'])
y = df['cyberbullying_type']

print("TF-IDF shape:", X_tfidf.shape)

TF-IDF shape: (47656, 5000)


In [39]:
# Feature Selection
# Chi-Square Test
from sklearn.feature_selection import SelectKBest, chi2

chi2_selector = SelectKBest(chi2, k=2000)  # keep top 2000 features
X_chi2 = chi2_selector.fit_transform(X_tfidf, y)

print("Chi-Square reduced shape:", X_chi2.shape)

Chi-Square reduced shape: (47656, 2000)


In [45]:
"""
4. Model Selection
Research suitable algorithms for text classification:
Logistic Regression
Naive Bayes (MultinomialNB)
Support Vector Machine (SVM)
Random Forest / Gradient Boosting
Deep learning models (LSTM, BiLSTM, Transformer-based models)
"""

'\n4. Model Selection\nResearch suitable algorithms for text classification:\nLogistic Regression\nNaive Bayes (MultinomialNB)\nSupport Vector Machine (SVM)\nRandom Forest / Gradient Boosting\nDeep learning models (LSTM, BiLSTM, Transformer-based models)\n'

In [47]:
# ===============================
# IMPORTS
# ===============================
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

# ===============================
# LOAD DATA
# ===============================
df = pd.read_csv("https://raw.githubusercontent.com/ramizcihe/week4-cihe240058/refs/heads/main/cyberbullying_tweets.csv")  # Replace with your CSV path

# ===============================
# QUICK CHECK
# ===============================
print("Columns:", df.columns)
print("First 5 rows:\n", df.head())
print("Label distribution:\n", df['cyberbullying_type'].value_counts())

# ===============================
# BASIC CLEANING
# ===============================
# Remove duplicates
df = df.drop_duplicates()

# Remove rows with empty tweets
df = df[df['tweet_text'].str.strip() != ""]

# ===============================
# FEATURE & LABEL
# ===============================
X = df['tweet_text']                  # Features
y = df['cyberbullying_type']          # Target labels

# ===============================
# LABEL ENCODING
# ===============================
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
print("Classes:", label_encoder.classes_)

# ===============================
# TRAIN-TEST SPLIT
# ===============================
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# ===============================
# TEXT VECTORIZATION
# ===============================
tfidf = TfidfVectorizer(
    max_features=5000,      # limit to top 5000 words
    stop_words='english'
)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# ===============================
# READY FOR MODELING
# ===============================
print("Shape of X_train:", X_train_tfidf.shape)
print("Shape of X_test:", X_test_tfidf.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)


Columns: Index(['tweet_text', 'cyberbullying_type'], dtype='object')
First 5 rows:
                                           tweet_text cyberbullying_type
0  In other words #katandandre, your food was cra...  not_cyberbullying
1  Why is #aussietv so white? #MKR #theblock #ImA...  not_cyberbullying
2  @XochitlSuckkks a classy whore? Or more red ve...  not_cyberbullying
3  @Jason_Gio meh. :P  thanks for the heads up, b...  not_cyberbullying
4  @RudhoeEnglish This is an ISIS account pretend...  not_cyberbullying
Label distribution:
 cyberbullying_type
religion               7998
age                    7992
gender                 7973
ethnicity              7961
not_cyberbullying      7945
other_cyberbullying    7823
Name: count, dtype: int64
Classes: ['age' 'ethnicity' 'gender' 'not_cyberbullying' 'other_cyberbullying'
 'religion']
Shape of X_train: (38124, 5000)
Shape of X_test: (9532, 5000)
Shape of y_train: (38124,)
Shape of y_test: (9532,)


In [50]:
# ===============================
# IMPORTS
# ===============================
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.utils import to_categorical

# ===============================
# LOAD DATA
# ===============================
df = pd.read_csv("https://raw.githubusercontent.com/ramizcihe/week4-cihe240058/refs/heads/main/cyberbullying_tweets.csv")  # Replace with your CSV path

# ===============================
# BASIC CLEANING
# ===============================
df = df.drop_duplicates()
df = df[df['tweet_text'].str.strip() != ""]

# ===============================
# LABEL ENCODING
# ===============================
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['cyberbullying_type'])
num_classes = len(label_encoder.classes_)
y_categorical = to_categorical(y)  # For multi-class classification

# ===============================
# TRAIN-TEST SPLIT
# ===============================
X_train, X_test, y_train, y_test = train_test_split(
    df['tweet_text'], y_categorical,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# ===============================
# TOKENIZATION & PADDING
# ===============================
max_words = 10000     # Max number of words to keep in tokenizer
max_len = 100         # Max length of sequences (padded)

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post', truncating='post')

# ===============================
# LSTM / BiLSTM MODEL
# ===============================
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Bidirectional(LSTM(64, return_sequences=False)))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# ===============================
# TRAINING
# ===============================
history = model.fit(
    X_train_pad, y_train,
    epochs=5,
    batch_size=64,
    validation_split=0.1
)

# ===============================
# EVALUATION
# ===============================
loss, accuracy = model.evaluate(X_test_pad, y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
537/537 ━━━━━━━━━━━━━━━━━━━━ 151s 272ms/step - accuracy: 0.6139 - loss: 0.9353 - val_accuracy: 0.8041 - val_loss: 0.4375
Epoch 2/5
537/537 ━━━━━━━━━━━━━━━━━━━━ 196s 262ms/step - accuracy: 0.8463 - loss: 0.3626 - val_accuracy: 0.8416 - val_loss: 0.4046
Epoch 3/5
537/537 ━━━━━━━━━━━━━━━━━━━━ 165s 306ms/step - accuracy: 0.8891 - loss: 0.2794 - val_accuracy: 0.8408 - val_loss: 0.4174
Epoch 4/5
537/537 ━━━━━━━━━━━━━━━━━━━━ 173s 321ms/step - accuracy: 0.9133 - loss: 0.2267 - val_accuracy: 0.8167 - val_loss: 0.4607
Epoch 5/5
537/537 ━━━━━━━━━━━━━━━━━━━━ 148s 275ms/step - accuracy: 0.9217 - loss: 0.1974 - val_accuracy: 0.8280 - val_loss: 0.5089
298/298 ━━━━━━━━━━━━━━━━━━━━ 11s 35ms/step - accuracy: 0.8243 - loss: 0.5157
Test Accuracy: 82.49%


In [52]:
print(df.columns)

Index(['tweet_text', 'cyberbullying_type'], dtype='object')


In [48]:
"""
5. Dataset Splitting
Split into training (70%), validation (15%), and test (15%) sets.
Use stratified splitting to maintain class proportion.
"""

'\n5. Dataset Splitting\nSplit into training (70%), validation (15%), and test (15%) sets.\nUse stratified splitting to maintain class proportion.\n'